In [0]:
%run "/Repos/pakkiraiah.kandi@outlook.com/databricks-practice/Formula1/Includes/configurations"


In [0]:
%run "/Repos/pakkiraiah.kandi@outlook.com/databricks-practice/Formula1/Includes/common_functions"

In [0]:
read_formula_df=spark.read.parquet(f"{presentation_folder_path}/formula1_results")


In [0]:
demo_df=read_formula_df.filter("race_year=='2020'").orderBy(read_formula_df["points"].desc())


In [0]:
display(demo_df)

In [0]:
#Aggregator functions

demo_df.select(count("*")).show()
demo_df.select(count("race_name")).show()
demo_df.select(countDistinct("race_name")).show()
demo_df.select(mean("points")).show()
demo_df.select(sum("points")).show()
demo_df.select(max("points")).show()

In [0]:
demo_df.filter("driver_name=='Lewis Hamilton'").select(sum("points").alias("total_points"),countDistinct("race_name").alias("number_of_races")).show()


In [0]:
#Group by functons

demo_groupby_df=read_formula_df.groupBy("race_year","driver_name").agg(sum("points").alias("total_points")
                                    ,countDistinct("race_name").alias("number_of_races"))

display(demo_groupby_df)




In [0]:
#Window Functions

from pyspark.sql.window import Window
from pyspark.sql.functions import rank
demo_window_spec=Window.partitionBy("race_year").orderBy(desc("total_points"))

demo_groupby_df=demo_groupby_df.withColumn("rank",rank().over(demo_window_spec))
#.filter("race_year==2020")

display(demo_groupby_df)


In [0]:
#Ranking the drivers based on their total points using rank
driverSpec=Window.partitionBy("race_year").orderBy(desc("total_points"))


In [0]:
#Driver standings
driverStandings=read_formula_df\
    .groupBy("race_year","driver_name","driver_nationality","constructor_name")\
        .agg(sum("points").alias("total_points"),\
            count(when(col("position")=="1",1).otherwise(0)).alias("Wins"))

driverRankSpec=Window.partitionBy("race_year").orderBy(desc("total_points"),desc("Wins"))

driver_standings_df=driverStandings.withColumn("rank",rank().over(driverRankSpec))



display(driver_standings_df)

In [0]:
#Writting driver standings into presentation layer
driver_standings_df.write.mode("overwrite").format("parquet").parquet(f"{presentation_folder_path}/driver_standings")

In [0]:
#Read driverStandings from presentation layer
driverStandings=spark.read.parquet(f"{presentation_folder_path}/driver_standings")
display(driverStandings)

In [0]:
#Constructors standings
constructorSpec=Window.partitionBy("race_year").orderBy(desc("total_points"),desc("Wins"))

constructors_standig_df=read_formula_df\
    .groupBy("race_year","constructor_name")\
        .agg(sum("points").alias("total_points"),\
        count(when(col("position")=="1",1).otherwise(0)).alias("Wins"))
constructors_standig_df=constructors_standig_df.withColumn("rank",rank().over(constructorSpec))

display(constructors_standig_df.filter("race_year==2020"))

In [0]:
#Writting Team standings into presentation layer
constructors_standig_df.write.mode("overwrite").format("parquet").parquet(f"{presentation_folder_path}/constructors_standig")

In [0]:
#Read team standing from presentation layer
team_standing_df=spark.read.parquet(f"{presentation_folder_path}/constructors_standig")
display(team_standing_df.filter("race_year==2020"))